# Sub-Numpy
We will create our own implementation of a few functionalities supported by the NumPy
Library. We will call our implementation SNumPy (for Sub-NumPy). SNumPy will be the name of the
class you implement, and we will refer to it by the shorthand “snp” from here on

In [109]:
# Create SNumPy class that will hold our methods for data manipulation methods,  not using numpy
# Including snp.ones(Int), snp.zeros(Int), snp.reshape(array, (row, column)), snp.shape(array), snp.append(array1, array2)
# snp.get(array, (row, column)), snp.add(array1, array1), snp.subtract(array1, array1), snp.dotproduct(array1, array1)

class SNumPy:
    """
    SNumPy class for basic array manipulations.
    """

    @staticmethod
    def ones(n, m=None):
        """
        Return an array of ones in given shape (n,m).
        """
        if m is None:
            return [1 for i in range(n)]
        else:
            return [[1 for i in range(m)] for j in range(n)]


    @staticmethod
    def zeros(n, m=1):
        """
        Return an array of zeros in given shape (n,m).
        """
        if m is None:
            return [0 for i in range(n)]
        else:
            return [[0 for i in range(m)] for j in range(n)]

    @staticmethod
    def reshape(array, shape):
        """
        Return an array containing the same data with a new shape.
        """
        column, row = shape
        new_array = []
        for i in range(row):
            new_array.append(array[i*column:(i+1)*column])
        return new_array

    @staticmethod
    def shape(array):
        """
        Return the shape of an array.
        """
        # Check if it's a vector (1D array)
        if not array or not isinstance(array[0], list):
            return (len(array),)
        # Else, it's a matrix (2D array)
        else:
            return (len(array), len(array[0]))

    @staticmethod
    def append(array1, array2):
        """
        Returns a new vector/matrix that is the combination of the two input vectors/matrices.
        """
        # Vectors and Matrices cant be combined
        if len(SNumPy.shape(array1)) != len(SNumPy.shape(array2)):
            raise ValueError(f"Vectors and Matrices cannot be combined, array1: {SNumPy.shape(array1)}, array2: {SNumPy.shape(array2)}")
        
        return array1 + array2

    @staticmethod
    def get(array, index):
        """
        Return the element at the given index.
        """
        column, row = index  # Unpack the index tuple
        return array[row][column]

    @staticmethod
    def add(array1, array2):
        """
        Return the element-wise sum of two arrays.
        """
        try:
            if SNumPy.shape(array1) != SNumPy.shape(array2):
                raise ValueError("Arrays must be the same size")
        except ValueError as e:
            print(e)
            return None

        # Using zip() will parrallelize the addition of the arrays
        return [
            [
                cell1 + cell2 
                for cell1, cell2 in zip(row1, row2)
            ] 
            for row1, row2 in zip(array1, array2)
        ]

    @staticmethod
    def subtract(array1, array2):
        """
        Return the element-wise difference of two arrays.
        """
        try:
            if SNumPy.shape(array1) != SNumPy.shape(array2):
                raise ValueError("Arrays must be the same size")
        except ValueError as e:
            print(e)
            return None
        
        # Check if they are vectors (1D arrays)
        if len(SNumPy.shape(array1)) == 1:
            return [cell1 - cell2 for cell1, cell2 in zip(array1, array2)] # Using zip() will parrallelize the subtraction of the arrays
        # Else, they are matrices (2D arrays)
        else:
            return [
                [cell1 - cell2 for cell1, cell2 in zip(row1, row2)]
                for row1, row2 in zip(array1, array2)
            ]

    @staticmethod
    def dotproduct(array1, array2):
        """
        Check if its vectors or matrices, then return the dot product.
        """
        # Check if its vectors
        if len(SNumPy.shape(array1)) == 1 and len(SNumPy.shape(array2)) == 1:
            # Check if the vectors are the same size
            try:
                if SNumPy.shape(array1) != SNumPy.shape(array2):
                    raise ValueError("Vectors must be the same size")
            except ValueError as e:
                print(e)
                return None

            return sum([array1[i] * array2[i] for i in range(len(array1))])
        
        # Else do matrix multiplication
        else:
            # Check if the columns of array1 are the same size as the rows of array2
            try:
                if SNumPy.shape(array1)[1] != SNumPy.shape(array2)[0]:
                    raise ValueError(f"Array1's columns '{SNumPy.shape(array1)[1]}'  must be the same size as Array2's rows '{SNumPy.shape(array2)[0]}'")
            except ValueError as e:
                print(e)
                return None

            return [
                [
                    sum(
                        [array1[i][k] * array2[k][j] 
                        for k in range(len(array1[0]))]
                    ) 
                    for j in range(len(array2[0]))
                ] 
                for i in range(len(array1))
            ]
        
    @staticmethod
    def scalar_multiply(array, scalar):
        """
        Return the scalar product of an array.
        """
        # Check if its a vector
        if len(SNumPy.shape(array)) == 1:
            return [element * scalar for element in array]
        # Else, its a matrix
        else:
            return [[element * scalar for element in row] for row in array]
        
    @staticmethod
    def aug_matrix(matrix, vector):
        """
        Append a vector as a new column to the right of a matrix.
        """
        if len(matrix) != len(vector):
            raise ValueError("Length of vector must match the number of rows in the matrix")
        
        augmented_matrix = [row + [vector[i]] for i, row in enumerate(matrix)]
        return augmented_matrix
            
 

Gaussian Elimination adopted from the code found in this video, modified to work with list and not arrays as well as using our own SNumPy methods. (StudySession, 2023)

StudySession (Director). (2023, February 11). Gauss Elimination With Partial Pivoting In Python | Numerical Methods. https://www.youtube.com/watch?v=DiZ0zSzZj1g


In [145]:
def gaussianElimination(matrix, vector):
    """
    Solve a system of linear equations using Gaussian Elimination.

    This function applies Gaussian Elimination to solve the system of linear equations represented by the matrix equation Ax = B, where A is a square coefficient matrix, and B is a constant vector. 
    The function includes forward elimination with partial pivoting and backward substitution. 
    It checks for non-square matrices, singular matrices, and mismatched dimensions between the matrix and vector.

    Args:
        matrix : List[List[float]]
            Coefficient matrix (A), a 2D list representing the coefficients of the linear equations. Must be square (NxN).
        vector : List[float]
            Constant vector (B), a list representing the constant terms of the linear equations. Length must match the number of rows in 'matrix'.

    Returns:
        x : List[float]
            Solution vector (x) to the system Ax = B. Returns a list of float values representing the solution.

    Raises:
        ValueError
            If the matrix is not square, if the matrix is singular (determinant is zero), or if the dimensions of the matrix and vector do not match.
    """
    # Error checking for input dimensions
    try:
        if SNumPy.shape(vector) != SNumPy.shape(matrix[0]):
            raise ValueError(
                f"Vector must be the same size as the matrix's rows '{SNumPy.shape(matrix[0])}'")
    except ValueError as e:
        print(e)
        return None
    try:
        if SNumPy.shape(matrix[0]) != SNumPy.shape(matrix[1]):
            raise ValueError("Matrix must be a square matrix")
    except ValueError as e:
        print(e)
        return None

    # Create necessary variables
    n = len(vector)
    m = n - 1 
    x = SNumPy.zeros(n) # Initialize solution vector with zeros

    # Augmented matrix
    augmented_matrix = SNumPy.aug_matrix(matrix, vector)

    print(f"Augmented Matrix: \n {augmented_matrix}")

    # Forward elimination
    for i in range(n):
        # Partial pivoting for numerical stability (dealing with floating point errors)
        max_row = max(range(i, n), key=lambda r: abs(augmented_matrix[r][i]))
        try:
            # Check for singular matrix (no unique solution)
            if augmented_matrix[max_row][i] == 0.0:
                raise ValueError("Matrix is singular")
        except ValueError as e:
            print(e)
            return None
        
        augmented_matrix[i], augmented_matrix[max_row] = augmented_matrix[max_row], augmented_matrix[i]

        # Eliminate the entries below the current pivot
        for j in range(i + 1, n):
            scaling_factor = augmented_matrix[j][i] / augmented_matrix[i][i]
            augmented_matrix[j] = SNumPy.subtract(
                augmented_matrix[j], SNumPy.scalar_multiply(augmented_matrix[i], scaling_factor))

    print(f"Upper Triangular Matrix: \n {augmented_matrix}")

    # Backward substitution
    x[m] = augmented_matrix[m][n] / augmented_matrix[m][m]

    for i in range(m - 1, -1, -1):
        x[i] = augmented_matrix[i][n]
        for j in range(i + 1, n):
            x[i] = x[i] - augmented_matrix[i][j] * x[j]
        x[i] = x[i] / augmented_matrix[i][i]

    # Return the solution vector
    return x

In [146]:
import numpy as np

matrix = [[3, 2, -4], [2, 3, 3], [5, -3, 1]]
vector = [3, 15, 14]
solution = gaussianElimination(matrix, vector)
print(solution)

Augmented Matrix: 
 [[3, 2, -4, 3], [2, 3, 3, 15], [5, -3, 1, 14]]
Upper Triangular Matrix: 
 [[5, -3, 1, 14], [0.0, 4.2, 2.6, 9.399999999999999], [0.0, 0.0, -6.952380952380952, -13.904761904761903]]
[3.0, 1.0, 2.0]


In [111]:
# Test cases for SNumPy class

snp = SNumPy()
print(snp.ones(5))
print(snp.zeros(5))
print(snp.reshape([1, 2, 3, 4, 5, 6], (2, 3)))
print(snp.shape([[1, 2, 3], [4, 5, 6]]))
print(snp.append([[1, 2, 3]], [[4, 5, 6]]))
print(snp.get([[1, 2, 3], [4, 5, 6]], (1, 1)))
print(snp.add([[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]))
print(snp.subtract([[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]))
print(snp.dotproduct([[1, 2, 3]], [[4, 5, 6], [1, 2, 3], [7, 8, 9]]))
print(snp.scalar_multiply([[1, 2, 3], [4, 5, 6]], 2))

[1, 1, 1, 1, 1]
[[0], [0], [0], [0], [0]]
[[1, 2], [3, 4], [5, 6]]
(2, 3)
[[1, 2, 3], [4, 5, 6]]
5
[[2, 4, 6], [8, 10, 12]]
[[0, 0, 0], [0, 0, 0]]
[[27, 33, 39]]
[[2, 4, 6], [8, 10, 12]]
